# Bandas de Bollinger (Bollinger Band)
Acompanhar e identificar a variação dos preços baseado em médias mópveis e desvios padrões, sendo um indicador para medição de volatilidade.<br />
- São criadas duas "bandas" sendo as faixas de concentração dos preços dentro de 2 desvios padrões.
- Contribui para entender preços que estão atingindo áreas de subvalorização ou supervalorização.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
import plotly.subplots as pls
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Acao = "CPLE3.SA"
Periodo_Inicio = datetime.datetime(2019,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
Dados = yf.download(Acao, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados

### Análise dos Preços
Candlestick da série de preços do ativo.

In [ ]:
SeriePrecos = plg.Figure(plg.Scatter(x=Dados.index, y=Dados["Close", Acao]))
SeriePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(SeriePrecos, "Preços")

In [ ]:
CandlePrecos = plg.Figure(plg.Candlestick(x=Dados.index,
                                          open=Dados["Open", Acao],
                                          high=Dados["High", Acao],
                                          low=Dados["Low", Acao],
                                          close=Dados["Close", Acao]))
CandlePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(CandlePrecos, "Preços")

### Cálculo das Bandas
Banda Superior = Média Móvel Simples 20d + (2 x Desvio de 20 dias). <br />
Banda Inferior = Média Móvel Simples 20d - (2 x Desvio de 20 dias). <br />
A partir das bandas são possíveis identificar as áreas de Suporte e Resistência, sendo elas:
- Suporte: região onde os preços tem dificuldade para continuar caindo.
- Resistência: região onde os preços tem dificuldade de aumentar mais, indicando vendedores no controle do cenário.
- Quando as bandas são estreitas entendemos que a demanda e oferta podem estar em equilíbrio.

In [ ]:
JanelaDeTempo = 20

In [ ]:
MA20 = Dados["Close", Acao].rolling(window=JanelaDeTempo).mean().dropna()

In [ ]:
Desvio20 = Dados["Close", Acao].rolling(window=JanelaDeTempo).std().dropna()

In [ ]:
BandaSuperior = MA20 + 2 * Desvio20

In [ ]:
BandaInferior = MA20 - 2 * Desvio20

In [ ]:
SerieComMediaMovel = plg.Figure()
SerieComMediaMovel.add_trace(plg.Candlestick(x=Dados.index,
                                            open=Dados["Open", Acao],
                                            high=Dados["High", Acao],
                                            low=Dados["Low", Acao],
                                            close=Dados["Close", Acao],
                                            name="Preços"))
SerieComMediaMovel.add_trace(plg.Scatter(x=BandaSuperior.index,
                                         y=BandaSuperior,
                                         name="Superior",
                                            opacity=0.5))
SerieComMediaMovel.add_trace(plg.Scatter(x=MA20.index,
                                         y=MA20,
                                         name="MA 20d",
                                            opacity=0.5))
SerieComMediaMovel.add_trace(plg.Scatter(x=BandaInferior.index,
                                         y=BandaInferior,
                                         name="Inferior",
                                            opacity=0.5))
Plot_Show(SerieComMediaMovel, "Preços & Bandas de Bollinger")